# Introductory Analysis

In [1]:
from pathlib import *
import sqlalchemy as sa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import axelrod as axl
from datetime import date
from pandas.plotting import table
%matplotlib inline

Size of Database as of writing: 

In [2]:
today_date = date.today().strftime("%d_%m_%Y")
database_management_sys = sa.create_engine("sqlite:///../database-code/data/se/main.db")
connect_dbms_to_db = database_management_sys.connect()
se_folk_thm_data = pd.read_sql("folk_theorem_experiment", connect_dbms_to_db)

database_size_path = Path("../database-code/data/se/" + today_date + "/")
database_size_path.mkdir(parents=True, exist_ok=True)
p = database_size_path / "entries-in-database.txt"
q = database_size_path / "number-of-tournaments.txt"
with open(str(p), "w") as num_of_entries_file:
    num_of_entries_file.write(str(len(se_folk_thm_data)))
with open(str(q), "w") as num_of_tournaments_file:
    num_of_tournaments_file.write(str(max(se_folk_thm_data["tournament_player_set"])))
len(se_folk_thm_data), max(se_folk_thm_data["tournament_player_set"])

(825700, 159)

In [3]:
se_folk_thm_data.head()

,experiment_number,number_of_players,tournament_player_set,player_strategy_name,is_long_run_time,is_stochastic,memory_depth_of_strategy,prob_of_game_ending,payoff_matrix,num_of_repetitions,num_of_equilibria,nash_equilibria,least_prob_of_defection,greatest_prob_of_defection,noise,warning_message
0,0,2,0,Inverse,False,True,Inf,0.001000,3. 0.99159429 1.03362284 1.,500,3.0,1. 0. 1. 0. ...,0.0,1.0,0.0,None
1,0,2,0,Defector,False,False,0,0.001000,3. 0.99159429 1.03362284 1.,500,3.0,1. 0. 1. 0. ...,0.0,1.0,0.0,None
2,1,2,0,Inverse,False,True,Inf,0.011081,3. 0.95530245 1.1787902 1.,500,3.0,1. 0. 1. 0. ...,0.0,1.0,0.0,None
3,1,2,0,Defector,False,False,0,0.011081,3. 0.95530245 1.1787902 1.,500,3.0,1. 0. 1. 0. ...,0.0,1.0,0.0,None
4,2,2,0,Inverse,False,True,Inf,0.021162,3. 0.9212484 1.31500641 1.,500,3.0,1. 0. 1. 0. ...,0.0,1.0,0.0,None


### Summary Statistics

In [ ]:
summary_stats_path = Path("../../images/folk_thm/initial_analysis/")
summary_stats_path.mkdir(parents=True, exist_ok=True)
summary_stats = se_folk_thm_data.describe()
p = summary_stats_path / "summary_stats.html"
table_html = summary_stats.to_html(str(p))

The average number of players per tournament in the experiment is 5 players.

The probabilities of the game ending is centred around p = 0.5.

The average number of Nash equilibria calculated from the payoff matrices is 2, with the majority of games having 1 - 3 equilibria. However, what is interesting here is that at least one game obtained 39 equilibria (as can be seen from the table above). A further search regarding this will be executed later.

The mean for the least probability of defection in any one game is 0.342275 and for the greatest probability of defection it is 0.459722. However, both probabilities have yielded a value of 0 and 1 over the experiment. 

### Strategy Statistics

In [ ]:
strategy_data = se_folk_thm_data[["tournament_player_set", "player_strategy_name", "is_stochastic", "is_long_run_time", "memory_depth_of_strategy"]].drop_duplicates()
strategy_data.head()

In [ ]:
strategy_counts = strategy_data["player_strategy_name"].value_counts()
strategy_counts.head(), strategy_counts.tail()

In [ ]:
len(strategy_counts[strategy_counts == 1]), len(strategy_data["player_strategy_name"].drop_duplicates())

Excluding the defector, the strategy which appeared the most (9 times) is ZD-GEN-2: 0.125, 0.5, 3; followed closely by Tideman and Chieruzzi with 7 times. On the other hand 38 out of the 200 strategies executed in this experiment appeared only once. Next, these counts will be plotted with regards to the players characteristics to see if there is any overarching bias that needs to be taken into account.

In [ ]:
strategy_data["is_stochastic"].value_counts()

In [ ]:
strategy_data["is_long_run_time"].value_counts()

In [ ]:
strategy_data["memory_depth_of_strategy"].value_counts()

The majority of strategies participating in this experiment had either an infinite, one or no memory depth.

In [ ]:
strategy_counts_no_defect = strategy_counts.drop("Defector")
strategy_counts_no_defect.head(), len(strategy_counts_no_defect)

In [ ]:
labels = []
for player in strategy_counts_no_defect.index:
    stochastic_player = pd.Series(strategy_data[strategy_data["player_strategy_name"] == player]["is_stochastic"])
    stochastic_player.index = range(len(stochastic_player))
    stochastic_player = stochastic_player[0]
    labels.append(stochastic_player)

In [ ]:
len(labels)

In [ ]:
player_stochastic_dict = {"player_strategy_name" : strategy_counts_no_defect.index,
                            "strategy_counts" : strategy_counts_no_defect.values,
                            "is_stochastic" : labels}

player_stochastic_data = pd.DataFrame(player_stochastic_dict)

In [ ]:
player_stochastic_data.head()

In [ ]:
stochastic_player_data = player_stochastic_data[player_stochastic_data["is_stochastic"] == True]
deterministic_player_data = player_stochastic_data[player_stochastic_data["is_stochastic"] == False]
stochastic_player_data.head(), deterministic_player_data.head()

In [ ]:
len(stochastic_player_data[stochastic_player_data["strategy_counts"] == 2])

In [ ]:
plot_path = summary_stats_path / "strategy_appearances.pdf"

stochastic_counts = stochastic_player_data["strategy_counts"].value_counts()
deterministic_counts = deterministic_player_data["strategy_counts"].value_counts()
xaxis = np.arange(len(label_for_xaxis))
for index in range(1, 10):
    if index not in stochastic_counts.index:
        stochastic_counts[index] = 0
    else:
        continue
stochastic_counts = stochastic_counts.sort_index()
for index in range(1, 10):
    if index not in deterministic_counts.index:
        deterministic_counts[index] = 0
    else:
        continue
deterministic_counts = deterministic_counts.sort_index()

width_of_bars = 0.25
xaxis_labels = list(range(1, 10))
xaxis = np.arange(len(xaxis_labels))

graph = plt.figure()
axes = graph.add_subplot(1, 1, 1)
axes.set_xlabel("Number of appearances")
axes.set_ylabel("Number of strategies")
axes.set_xticks(xaxis)
axes.set_xticklabels(xaxis_labels)
axes.bar(xaxis - width_of_bars/2, list(stochastic_counts.values), width_of_bars, label="Stochastic Strategy")
axes.bar(xaxis + width_of_bars/2, list(deterministic_counts.values), width_of_bars, label="Deterministic Strategy")
axes.legend(title="Stochastic Strategy?")
graph.savefig(str(plot_path))

### Number of Nash Equilibria

In [4]:
nash_equilibria_data = se_folk_thm_data[["experiment_number", "number_of_players", "prob_of_game_ending", "num_of_equilibria", "nash_equilibria"]].drop_duplicates()
nash_equilibria_data.head()

,experiment_number,number_of_players,prob_of_game_ending,num_of_equilibria,nash_equilibria
0,0,2,0.001000,3.0,1. 0. 1. 0. ...
2,1,2,0.011081,3.0,1. 0. 1. 0. ...
4,2,2,0.021162,3.0,1. 0. 1. 0. ...
6,3,2,0.031242,3.0,1. 0. 1. 0. ...
8,4,2,0.041323,3.0,1. 0. 1. 0. ...


In [70]:
player_nash_eq = {}
for player_num in nash_equilibria_data["number_of_players"].drop_duplicates():
    player_nash_eq[str(player_num)] = nash_equilibria_data[nash_equilibria_data["number_of_players"] == player_num]["num_of_equilibria"]
    player_nash_eq[str(player_num)].index = range(len(nash_equilibria_data[nash_equilibria_data["number_of_players"] == player_num]["num_of_equilibria"]))
nash_data = pd.DataFrame(player_nash_eq)
nash_data = nash_data.drop("8", 1)
xaxis_labels = list(range(2, 8))
nash_eq_graph = plt.figure()
axes = nash_eq_graph.add_subplot(1, 1, 1)
axes.violinplot(nash_data)
nash_eq_graph.show()

C:\Users\sophi\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:2093: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
C:\Users\sophi\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
C:\Users\sophi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  # This is added back by InteractiveShellApp.init_path()
Error in callback <function flush_figures at 0x000001FE7593C8C8> (for post_execute):


KeyboardInterrupt: 

In [68]:
xaxis_labels

[2, 3, 4, 5, 6, 7]